# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('End_time.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   End_time           576 non-null    float64
 3   End_index          576 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 18.1 KB


In [3]:
y = dataset['End_time']
X = dataset.drop('End_time', axis=1)
print(X.shape, y.shape)

(576, 3) (576,)


In [4]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning - layers, neurons, activation function

In [6]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [7]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\Chathura Gamage\AppData\Local\Temp\ipykernel_9092\2480470563.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [9]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3],
                  first_layer_nodes = [2,3,4,5],
                  last_layer_nodes = [5, 6, 7, 8],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
8/8 [==============================] - 1s 2ms/step - loss: 153.4124 - mae: 11.1147
Epoch 2/500
8/8 [==============================] - 0s 2ms/step - loss: 130.1689 - mae: 10.2312
Epoch 3/500
8/8 [==============================] - 0s 2ms/step - loss: 109.9464 - mae: 9.3831
Epoch 4/500
8/8 [==============================] - 0s 2ms/step - loss: 91.8868 - mae: 8.5614
Epoch 5/500
8/8 [==============================] - 0s 2ms/step - loss: 76.1078 - mae: 7.7763
Epoch 6/500
8/8 [==============================] - 0s 2ms/step - loss: 62.4566 - mae: 7.0301
Epoch 7/500
8/8 [==============================] - 0s 2ms/step - loss: 51.0012 - mae: 6.3343
Epoch 8/500
8/8 [==============================] - 0s 2ms/step - loss: 41.2412 - mae: 5.6723
Epoch 9/500
8/8 [==============================] - 0s 2ms/step - loss: 33.0052 - mae: 5.0544
Epoch 10/500
8/8 [==============================] - 0s 2ms/step - loss: 26.1498 - mae: 4.4789
Epoch 11/500
8/8 [==============================] - 0s 2ms/step 

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [10]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=2, input_shape=(X_train.shape[1],), activation='softplus'))
 model.add(Dense(units=4, activation='softplus'))
 model.add(Dense(units=6, activation='softplus'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [12]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [30,40,50]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
14/14 [==============================] - 1s 2ms/step - loss: 0.0471 - mae: 0.2000
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0233 - mae: 0.1268
Epoch 3/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0135 - mae: 0.0846
Epoch 4/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0081 - mae: 0.0591
Epoch 5/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0053 - mae: 0.0474
Epoch 6/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0035 - mae: 0.0418
Epoch 7/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0025 - mae: 0.0382
Epoch 8/500
14/14 [==============================] - 0s 2ms/step - loss: 0.0019 - mae: 0.0350
Epoch 9/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0016 - mae: 0.0347
Epoch 10/500
14/14 [==============================] - 0s 1ms/step - loss: 0.0015 - mae: 0.0331
Epoch 11/500
14/14 [==============================] - 0s 1m